<a href="https://colab.research.google.com/github/sbc806/cpsc_545_project/blob/main/sc_mvae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Requires that repository be public
!git clone https://github.com/sbc806/scrna_mvae.git

In [3]:
!pip install geoopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.7 MB/s eta 0:00:00


In [4]:
!pip install scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=2986ffa4e610ca0718cfd6a1b5a8858654a709ef8a14b7e59c8374c8c8d181d5
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=4109bd00dd3696a9063fabb920e2121d355d6a1eee8e39640f2c37bb37d6179c
  Stored in directory: /root/.cache/pip/wheels/6a/aa/

In [5]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [6]:
!pip install anndata

In [7]:
!pip install -U loompy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for loompy: filename=loompy-3.0.7-py3-none-any.whl size=52018 sha256=eb32d89b05d15950ba213d7b188b429bf33b246dc02e8516f85b1740458508a6
  Stored in directory: /root/.cache/pip/wheels/2c/22/1f/792a4621bb631e538bf1c21feae9bbaa6b19fd6d6ab382d1fd
  Created wheel for numpy-groupies: filename=numpy_groupies-0.9.22-py3-none-any.whl size=25846 sha256=1684ef8917cdf590d1313537c2a75c674332acfde3776815f5c1b04692947284
  Stored in directory: /root/.cache/pip/wheels/5b/ce/73/9192b12bb05cc6a44ba159cb733270b1539004037302a42077
Successfully built loompy numpy-groupies


In [8]:
import sys as sys

print(sys.path)
# If running notebook outside of scrna_mvae directory, add it to the path
sys.path.append('/content/scrna_mvae/')

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [9]:
import argparse
import datetime
import os

import torch

from mt.data import create_dataset
from mt.mvae import utils
from mt.mvae.models import Trainer
from mt.utils import str2bool
from mt.mvae.models.nb_vae import NBVAE
import numpy as np

import pandas as pd
import scanpy as sc

In [10]:
# The path to the datasets
# Assumed all datasets have the same base path
dataset_path = '/content/drive/MyDrive/mvae_research/datasets'

# Keys are the dataset names
# Values are the subdirectories and the file containing the dataset
datasets_paths = {"hela": "revelio_paper/hela/hela_select.loom",

                "DataSetS2": "cyclum_paper/DataSetS2.h5ad",

                "velocity_human": "deepcycle_paper/human/velocity_human_fibroblast.h5ad",
                "velocity_mouse": "deepcycle_paper/mouse/velocity_mouse.h5ad",

                "melanoma": "cyclum_paper/melanoma/melanoma_single_cell_revised_v2.h5ad",
                "melanoma_processed": "cyclum_paper/melanoma/melanoma_single_cell_revised_v2_processed.h5ad",
                "melanoma_78": "cyclum_paper/melanoma/melanoma_78.h5ad",

                "mouse_retina": "tricycle_paper/mouse_retina/mouse_retina_horizontal.h5ad",

                "ovarian_epithelial": "ovarian_cancer/local.h5ad",
                "ovarian_epithelial_raw": "ovarian_cancer/epithelial_count_added.h5ad",
                "ovarian_epithelial_cycling": "ovarian_cancer/ovarian_epithelial_cycling.h5ad",
                "ovarian_epithelial_cycling_raw": "ovarian_cancer/ovarian_epithelial_cycling.h5ad",

                "myeloid": "immune_cell_dataset/myeloid/CountAdded_PIP_myeloid_object_for_cellxgene.h5ad",
                "b_cells": "immune_cell_dataset/b_cells/CountAdded_PIP_B_object_for_cellxgene.h5ad",
                "t_cells": "immune_cell_dataset/CountAdded_PIP_global_object_for_cellxgene.h5ad",
                "immune_global": "immune_cell_dataset/CountAdded_PIP_global_object_for_cellxgene.h5ad",
                "myeloid_subset_1": "immune_cell_dataset/myeloid/myeloid_subset_1.h5ad",
                }

In [59]:
# vary important to prevent NAN
# torch.set_default_dtype(torch.float64)

# model_name = 's2, s2'
# fixed_curvature = True

# components = utils.parse_components(model_name, fixed_curvature)

# h_dim = 32
# scalar_parametrization = False
# device = 'cpu'

# dataset = create_dataset('mnist', 100, '/Users/jding/work/mvae/data/')
# train_loader, test_loader = dataset.create_loaders()

# ==
from mt.data.umi_data import UMIVaeDataset
import anndata
# adata = anndata.read_loom('/Users/jding/work/project/scdata/hela_select.loom')

# dataset_name should be a key in datasets_paths
dataset_name = "velocity_human"
full_dataset_path = os.path.join(dataset_path, datasets_paths[dataset_name])
# save_path is the path to save the embeddings to
save_path = '/content/drive/MyDrive/mvae_research/mvae_results'

# True if the raw counts should be used rather than the processed X
use_raw = True
print("Dataset:", dataset_name)
if dataset_name == "hela":
  adata = anndata.read_loom(full_dataset_path)
  save_path = os.path.join(save_path, 'hela')

elif dataset_name == "DataSetS2":
  adata = sc.read_h5ad(full_dataset_path)
  # adata.X = adata.X.astype(np.double)
  # adata.var['primerid'].index = np.arange(0, adata.n_vars, 1)
  save_path = os.path.join(save_path, 'DataSetS2')
  if use_raw:
    adata.X = adata.layers['X_rawCount']

elif dataset_name == "velocity_human":
  # adata = sc.read_h5ad('/content/drive/MyDrive/mvae_research/datasets/velocity_anndata_human_fibroblast_DeepCycle_ISMARA.h5ad')
  adata = sc.read_h5ad(full_dataset_path)
  adata.var['gene_name'] = adata.var['Accession'].index
  save_path = os.path.join(save_path, 'cell_cycle_gene_regulation/velocity_human')
  if use_raw:
    adata.layers['X_original'] = adata.X
    adata.X = adata.layers['X_raw'].todense().astype(np.double)
  else:
    adata.X = adata.X.todense().astype(np.double)

elif dataset_name == "velocity_mouse":
  adata = sc.read_h5ad(full_dataset_path)
  adata.var['gene_name'] = [gene.upper() for gene in adata.var['Accession'].index]
  save_path = os.path.join(save_path, 'cell_cycle_gene_regulation/velocity_mouse')
  if use_raw:
    adata.obsm['X_original'] = adata.X
    adata.X = adata.obsm['X_raw'].todense().astype(np.double)
  else:
    adata.X = adata.X.todense().astype(np.double)

elif dataset_name == "melanoma":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'cyclum_paper/melanoma')

elif dataset_name == "melanoma_processed":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'cyclum_paper/melanoma_processed')

elif dataset_name == "melanoma_78":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'cyclum_paper/melanoma_78')

elif dataset_name == "mouse_retina":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, "tricycle_paper/mouse_retina")

elif dataset_name == "ovarian_epithelial_raw":
  adata = sc.read_h5ad(full_dataset_path)
  adata.obsm['X_original'] = adata.X.astype(np.double)
  adata.X = adata.obsm['X_raw'].astype(np.double)
  save_path = os.path.join(save_path, 'ovarian/ovarian_epithelial')

elif dataset_name == "ovarian_epithelial":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'ovarian/ovarian_epithelial')

elif dataset_name == "ovarian_epithelial_cycling":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'ovarian/ovarian_epithelial_cycling')

elif dataset_name == "ovarian_epithelial_cycling_raw":
  adata = sc.read_h5ad(full_dataset_path)
  adata.obsm['X_original'] = adata.X.astype(np.double)
  adata.X = adata.obsm['X_raw'].astype(np.double)
  save_path = os.path.join(save_path, 'ovarian/ovarian_epithelial_cycling')

elif dataset_name == "myeloid":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'immune_cell_dataset/myeloid')

elif dataset_name == "b_cells":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'immune_cell_dataset/b_cells')

elif dataset_name == 't_cells':
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'immune_cell_dataset/t_cells')

elif dataset_name == "immune_global":
  adata = sc.read_h5ad(full_dataset_path)
  save_path = os.path.join(save_path, 'immune_cell_dataset/global')

elif dataset_name == "myeloid_subset_1":
  adata = sc.read_h5ad(full_dataset_path)
  adata.var['gene_name'] = adata.var.index
  save_path = os.path.join(save_path, 'immune_cell_dataset/myeloid_subset_1')
  if use_raw:
    adata.obsm['X_original'] = adata.X.astype(np.double)
    adata.X = adata.layers['counts'].astype(np.double)

Dataset: velocity_human


In [60]:
adata

AnnData object with n_obs × n_vars = 5367 × 10789
    obs: 'Clusters', '_X', '_Y', 'sample', 'n_genes', 'percent_mito', 'n_counts', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'S_score', 'G2M_score', 'phase', 'clusters_gradients', 'leiden', 'cell_cycle_theta'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'n_cells', 'gene_count_corr', 'gene_name'
    uns: 'ISMARA_activities', 'clusters_gradients_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'X_raw', 'ambiguous', 'matrix', 'spliced', 'unspliced', 'X_original'
    obsp: 'connectivities', 'distances'

Make sure to rearrange the genes so that the cycling genes are placed first before creating the training and test datasets.

The function for doing so is at the bottom of the notebook.

Create the training and test datasets

In [ ]:
# Does y actually get used in the evaluation?
if dataset_name == "hela":
  x = adata.X.todense().astype(np.double)
  y = adata.obs['batch'].rank(method='dense').astype(int)

elif dataset_name == "DataSetS2":
  x = adata.X.astype(np.double)
  y = adata.obs['cellline'].rank(method='dense').astype(int)

elif dataset_name == "velocity_human":
  x = adata.X
  y = adata.obs['Clusters'].rank(method='dense').astype(int)

elif dataset_name == "velocity_mouse":
  x = adata.X
  y = adata.obs['Clusters'].rank(method='dense').astype(int)

elif dataset_name == "melanoma":
  x = adata.X
  y = adata.obs['malignant(1=no,2=yes,0=unresolved)']

elif dataset_name == "melanoma_78":
  x = adata.X
  y = adata.obs['cell_id'].rank(method='dense').astype(int)

elif dataset_name == "melanoma_processed":
  x = adata.X
  y = adata.obs['malignant(1=no,2=yes,0=unresolved)']

elif dataset_name == "ovarian_epithelial":
  x = adata.X.todense().astype(np.double)
  y = adata.obs['cluster_label'].rank(method='dense').astype(int)

elif dataset_name == "ovarian_epithelial_cycling_raw":
  x = adata.X.todense().astype(np.double)
  y = adata.obs['cluster_label'].rank(method='dense').astype(int)

elif dataset_name == "myeloid_subset_1":
  x = adata.X.todense().astype(np.double)
  y = adata.obs['Manually_curated_celltype'].rank(method='dense').astype(int)

# Create the dataset and separate it into training and test sets
in_dim = x.shape[1]
print(f"in_dim = {in_dim}")
dataset = UMIVaeDataset(batch_size=128, in_dim=in_dim)
print(f"dataset.in_dim = {dataset.in_dim}")
train_loader = dataset.create_loaders(x, y)
test_loader = train_loader

for i, data in enumerate(train_loader):
    print((data[0].shape))

in_dim = 31897
dataset.in_dim = 31897
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([128, 31897])
torch.Size([

Create mask

In [ ]:
import pandas as pd

mask_cyc = np.zeros(adata.n_vars)

# The key for the gene names
if dataset_name == "hela":
  gene_names = 'gene_symbols'
elif dataset_name == "DataSetS2":
  gene_names = 'primerid'
elif dataset_name == "velocity_human":
  gene_names = 'gene_name'
elif dataset_name == "velocity_mouse":
  gene_names = 'gene_name'
elif dataset_name == "melanoma":
  gene_names = 'gene_name'
elif dataset_name == "melanoma_78":
  gene_names = 'gene_name'
elif dataset_name == "ovarian_epithelial":
  gene_names = 'gene_name'
elif dataset_name == "ovarian_epithelial_cycling_raw":
  gene_names = 'gene_name'
elif dataset_name == "myeloid":
  gene_names = 'gene_name'
elif dataset_name == "myeloid_subset_1":
  gene_names = 'gene_name'

# cycle_gene = pd.read_csv('/Users/jding/work/project/scdata/cycle_gene.txt', header=None, sep='\t')
cycle_gene = pd.read_csv('/content/drive/MyDrive/mvae/data/scdata/cycle_gene.txt', header=None, sep='\t')

# Obtain the cell cycling genes present in the cell x gene matrix
cycle_gene = cycle_gene[0][cycle_gene[0].isin(adata.var[gene_names])]

# A mask that is 1 where the gene is a cell cycling gene and 0 otherwise
mask_cyc[adata.var[gene_names].isin(cycle_gene)] = 1

# A mask that is 0 where the gene is a cell cycling gene and 1 otherwise
mask_all = np.ones(adata.n_vars)
mask_all[adata.var[gene_names].isin(cycle_gene)] = 0

num_gene = len(mask_cyc)
# Weight seems like currently not used
weight = (1 + np.array((num_gene - mask_cyc.sum(), num_gene - mask_all.sum()))) / num_gene

# mask = torch.tensor([mask_cyc * weight[0], mask_all * weight[1]])
# Original mask for s2, s2, way nb_vae is right now is that need row for each component
mask = torch.tensor([mask_cyc, mask_all])
# mask = torch.tensor([mask_cyc, mask_cyc, mask_all])

# 714 cycling genes
# mask = torch.tensor([np.ones(adata.n_vars), np.ones(adata.n_vars)])

In [ ]:
# vary important to prevent NAN
torch.set_default_dtype(torch.float64)

model_name = 'h2, e10'
fixed_curvature = True

components = utils.parse_components(model_name, fixed_curvature)

h_dim = 32
scalar_parametrization = False
device = 'cpu'

# dataset = create_dataset('mnist', 100, '/Users/jding/work/mvae/data/')
# train_loader, test_loader = dataset.create_loaders()

# ==

In [ ]:
print(model_name)
print(mask.shape)
print(mask_cyc)
print(mask_all)

h2, e10
torch.Size([2, 31897])
[1. 1. 1. ... 0. 0. 0.]
[0. 0. 0. ... 1. 1. 1.]


Run the model

In [ ]:
# Make sure input number and hyperparameter number is correct
input_number = 3
hyperparameter_number = 1

input_dir = "input_" + str(input_number)
input_path = os.path.join(save_path, input_dir)
if not os.path.exists(input_path):
  print("Making " + input_path)
  os.mkdir(input_path)

hyperparameter_dir = "hyperparameter_" + str(hyperparameter_number)
hyperparameter_path = os.path.join(input_path, hyperparameter_dir)
if not os.path.exists(hyperparameter_path):
  print("Making " + hyperparameter_path)
  os.mkdir(hyperparameter_path)

Making /content/drive/MyDrive/mvae_research/mvae_results/ovarian/ovarian_epithelial_cycling/input_3
Making /content/drive/MyDrive/mvae_research/mvae_results/ovarian/ovarian_epithelial_cycling/input_3/hyperparameter_1


In [ ]:
# =====
model = NBVAE(h_dim=h_dim,
              components=components,
              mask=mask,
              dataset=dataset,
              scalar_parametrization=scalar_parametrization).to(device)

cur_time = datetime.datetime.utcnow().isoformat()
chkpt_dir = f"./chkpt/vae-{'mnist'}-{model_name}-{cur_time}"

trainer = Trainer(model,
                  img_dims=[10,10],
                  chkpt_dir=chkpt_dir,
                  train_statistics=0,
                  show_embeddings=False,
                  export_embeddings=False,
                  test_every=0)

optimizer = trainer.build_optimizer(learning_rate=0.001, fixed_curvature=True)

betas = utils.linear_betas(1.0,
                           1.0,
                           end_epoch=1,
                           epochs=200)

# Originally, max_epochs=500
trainer.train_stopping(optimizer=optimizer,
                       train_data=train_loader,
                       eval_data=test_loader,
                       warmup=20,
                       lookahead=10,
                       betas=betas,
                       likelihood_n=200,
                       max_epochs=500)

use_log = True
if use_log:
  a = trainer.model(torch.log1p(torch.tensor(x)))
else:
  a = trainer.model(torch.tensor(x))
b = a[1]
bb = b.detach().numpy()

np.savetxt(os.path.join(hyperparameter_path, f'{model_name}_all_encode_v63.txt'), bb)
reconstructed_x = trainer.model.decode(a[1])
reconstructed_x = reconstructed_x[1].detach().numpy()
print(reconstructed_x.shape)
torch.save(model.state_dict(), os.path.join(hyperparameter_path, model_name))

	TrainEpoch 0:	{'bce': -11713.169934176192, 'kl': 66.86141033848862, 'elbo': -11780.03134451467, 'll': 0.0, 'mi': 0.0, 'cov_norm': 0.0, 'beta': 1.0, 'comp_000_h2/curvature': -1.0, 'comp_001_e10/curvature': 0.0}
	TrainEpoch 1:	{'bce': -10133.25290661171, 'kl': 29.08170421030118, 'elbo': -10162.334610822012, 'll': 0.0, 'mi': 0.0, 'cov_norm': 0.0, 'beta': 1.0, 'comp_000_h2/curvature': -1.0, 'comp_001_e10/curvature': 0.0}
	TrainEpoch 2:	{'bce': -10059.42711520845, 'kl': 27.01427656320849, 'elbo': -10086.441391771657, 'll': 0.0, 'mi': 0.0, 'cov_norm': 0.0, 'beta': 1.0, 'comp_000_h2/curvature': -1.0, 'comp_001_e10/curvature': 0.0}
	TrainEpoch 3:	{'bce': -9993.301949351664, 'kl': 26.37328791864989, 'elbo': -10019.675237270314, 'll': 0.0, 'mi': 0.0, 'cov_norm': 0.0, 'beta': 1.0, 'comp_000_h2/curvature': -1.0, 'comp_001_e10/curvature': 0.0}
	TrainEpoch 4:	{'bce': -9925.237433696257, 'kl': 26.983547349030093, 'elbo': -9952.220981045284, 'll': 0.0, 'mi': 0.0, 'cov_norm': 0.0, 'beta': 1.0, 'comp_0

In [ ]:
model_save_path = os.path.join(hyperparameter_path, model_name)
torch.save(model.state_dict(), model_save_path)

In [ ]:
for row in mask:
  print(row)
  print(torch.sum(row).item())

tensor([1., 1., 1.,  ..., 0., 0., 0.])
714.0
tensor([0., 0., 0.,  ..., 1., 1., 1.])
3831.0


In [ ]:
lines = ['Input Description',
         dataset_name,
         'Used Cycling.cancer.cell.1 only.',
         str(adata.X.shape),
         'Rearranged genes and made mask be [mask_cyc, mask_all].',
        'Used the raw counts.',
        'Used all genes.'

         ]

mask_description = "["
for i, row in enumerate(mask):
  mask_description = mask_description + str(torch.sum(row).item()) + ","
mask_description = mask_description + "]"
lines = lines + [mask_description]

if dataset_name == "velocity_human" or dataset_name == "velocity_mouse":
  if use_raw:
    lines = lines + ['Used the raw count matrix obtained from the GSE file.']
  else:
    lines = lines + ['Used the X included rather than the raw count matrix.']
with open(os.path.join(input_path, input_dir + '.txt'), 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

In [ ]:
lines = ['Hyperparameter Description',
         'Default hyperparameters.',
         f'Shape of the mask: {mask.shape}'
         'Commented out the constraint that z1 >= 0 in vae.py.'
         ]
if use_log:
  lines = lines + ['Used log.']
else:
  lines = lines + ["Did not use log."]
with open(os.path.join(hyperparameter_path, hyperparameter_dir+'.txt'), 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

In [ ]:
adata

Preprocessing

Data preprocessing

In [ ]:
# Data already normalized and log-transformed
# Select highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
# adata = adata[:, adata.var.highly_variable]
# print(adata.X.shape)
# Scale to unit variance and zero mean
sc.pp.scale(adata)

/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Get rid of batch effects

In [ ]:
!pip install bbknn

In [ ]:
import bbknn

bbknn.ridge_regression(adata, batch_key=['Chemistry'])

Alternative way to choose highly variable genes

In [ ]:
def process_ovarian_epithelial(ovarian_adata):
  print(ovarian_adata.X.dtype)
  sc.pp.highly_variable_genes(ovarian_adata, flavor="cell_ranger", n_top_genes=2000)

In [ ]:
if "ovarian_epithelial" in dataset_name:
  process_ovarian_epithelial(adata)
adata

float64


/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:540: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['hvg'] = {'flavor': flavor}


AnnData object with n_obs × n_vars = 38488 × 31897
    obs: 'percent.mt', 'percent.rb', 'doublet', 'sample', 'batch', 'patient_id', 'S.Score', 'G2M.Score', 'Phase', 'CC.Diff', 'RNA_snn_res.0.3', 'seurat_clusters', 'nCount_RNA', 'nFeature_RNA', 'doublet_score', 'cell_id', 'cell_type_super', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'cluster_label', 'CD8.Cytotoxic.module', 'CD8.Dysfunctional.module', 'CD8.Naive.module', 'CD8.Predysfunctional.module', 'CIN.module', 'CIN.responsive.Noncanonical.NFKB.targets.up.module', 'CIN.responsive.Noncanonical.NFKB.targets.down.module', 'Noncanonical.NFKB.regulators.up.module', 'Noncanonical.NFKB.regulators.down.module', 'Canonical.NFKB.regulators.module', 'Interferon.regulators.module', 'EMT.module', 'Inflammation.module', 'Motility.and.migration.module', 'ISG.module', 'SASP.module', 'Myofibroblast.module', 'Fibroblast.module', 'Mesenchymal.stem.cell.module', 'Pericytes.module', 'Smooth.muscle.cell.module', 'ER.stress.module', 'IFNg.signaling.module', 'An

In [ ]:
adata.var['gene_name']

gene.ids
ENSG00000243485    MIR1302-2HG
ENSG00000186092          OR4F5
ENSG00000238009     AL627309.1
ENSG00000239945     AL627309.3
ENSG00000241599     AL627309.4
                      ...     
ENSG00000276345     AC004556.1
ENSG00000277856     AC233755.2
ENSG00000275063     AC233755.1
ENSG00000271254     AC240274.1
ENSG00000268674        FAM231C
Name: gene_name, Length: 31897, dtype: category
Categories (31897, object): ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', ..., 'ZYG11A', 'ZYG11B', 'ZYX',
                             'ZZEF1']

In [ ]:
# Set any cell cycling genes to be included as a highly variable gene
add_in = True
highly_variable_genes = adata.var['highly_variable'].copy()
if add_in:
  all_cycle_gene = pd.read_csv('/content/drive/MyDrive/mvae/data/scdata/cycle_gene.txt', header=None, sep='\t')

  cycle_gene_present = all_cycle_gene[0][all_cycle_gene[0].isin(adata.var['gene_name'])]
  contained = adata.var['gene_name'].isin(cycle_gene_present)
  print(contained)
  highly_variable_genes[contained] = True

print(np.sum(highly_variable_genes))
adata = adata[:, highly_variable_genes]

In [ ]:
# Check if the sparse matrix contains infinity
contains_infinity = np.isinf(adata.obsm['X_original'].data).any()

# Print the result
if contains_infinity:
    print("The sparse matrix contains infinity.")
else:
    print("The sparse matrix does not contain infinity.")

The sparse matrix does not contain infinity.


Obtaining a subset of the cells

In [63]:
def obtain_subset_indices(adata, obs_key, desired_types):
  """
  Obtain the indices of cells that meet a criteria in observations.
  obs_key: The observation desired.
  desired_types: Which types of the observation should be kept.
  """
  subset_indices = adata.obs[obs_key] == desired_types[0]
  for i in range(1, len(desired_types)):
    subset_indices = subset_indices | (adata.obs[obs_key] == desired_types[i])
  return subset_indices

In [ ]:
subset_indices = obtain_subset_indices(adata, 'cluster_label', ['Cycling.cancer.cell.1',
                                                      'Cycling.cancer.cell.2',

                                                      ])

In [ ]:
# Obtain the subset
adata = adata[subset_indices]

In [ ]:
print(adata.X.shape)

(38488, 31897)


Obtaining a subset of the genes

In [16]:
def obtain_cycle_indices(adata, gene_name, cycle_gene_only=True):
  """
  Obtain the indices of either the cell cycling genes or the non-cell cycling genes.
  gene_name: A string that is the name of the variable in adata.var containing the gene names.
  cycle_gene_only: True if the value for the gene should be set to True if it is a cell cycling gene.
    False otherwise.
  """
  cycle_gene = pd.read_csv('/content/drive/MyDrive/mvae/data/scdata/cycle_gene.txt', header=None, sep='\t')
  print(f"Number of total cell cycle genes: {len(cycle_gene[0])}")
  print(cycle_gene[0])

  # Genes involved with the cell cycle present in adata
  cycle_gene = cycle_gene[0][cycle_gene[0].isin(adata.var[gene_name])]
  print(cycle_gene)

  if cycle_gene_only:
    desired_gene_indices = adata.var[gene_name].isin(cycle_gene)
    print(desired_gene_indices)
  else:
    desired_gene_indices = ~adata.var[gene_name].isin(cycle_gene)
  return desired_gene_indices

In [17]:
cycle_indices = obtain_cycle_indices(adata, 'gene_name', cycle_gene_only=True)

Number of total cell cycle genes: 800
0          ABCA7
1            ACD
2          ACYP1
3        ADAMTS1
4          ADCK2
         ...    
795        ZNF98
796        ZNFX1
797    ZNFX1-AS1
798         ZPBP
799      ZSCAN5A
Name: 0, Length: 800, dtype: object
0        ABCA7
1          ACD
2        ACYP1
3      ADAMTS1
4        ADCK2
        ...   
794     ZNF593
795      ZNF98
796      ZNFX1
798       ZPBP
799    ZSCAN5A
Name: 0, Length: 729, dtype: object
MIR1302-2HG    False
FAM138A        False
OR4F5          False
AL627309.1     False
AL627309.3     False
               ...  
AC141272.1     False
AC023491.2     False
AC007325.1     False
AC007325.4     False
AC007325.2     False
Name: gene_name, Length: 36601, dtype: bool


In [18]:
np.sum(cycle_indices)

729

In [ ]:
# Indexing if only want the cell cycling genes
adata = adata[:, cycle_indices]
adata

View of AnnData object with n_obs × n_vars = 149460 × 732
    obs: 'percent.mt', 'percent.rb', 'doublet', 'sample', 'batch', 'patient_id', 'S.Score', 'G2M.Score', 'Phase', 'CC.Diff', 'RNA_snn_res.0.3', 'seurat_clusters', 'nCount_RNA', 'nFeature_RNA', 'doublet_score', 'cell_id', 'cell_type_super', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'cluster_label', 'CD8.Cytotoxic.module', 'CD8.Dysfunctional.module', 'CD8.Naive.module', 'CD8.Predysfunctional.module', 'CIN.module', 'CIN.responsive.Noncanonical.NFKB.targets.up.module', 'CIN.responsive.Noncanonical.NFKB.targets.down.module', 'Noncanonical.NFKB.regulators.up.module', 'Noncanonical.NFKB.regulators.down.module', 'Canonical.NFKB.regulators.module', 'Interferon.regulators.module', 'EMT.module', 'Inflammation.module', 'Motility.and.migration.module', 'ISG.module', 'SASP.module', 'Myofibroblast.module', 'Fibroblast.module', 'Mesenchymal.stem.cell.module', 'Pericytes.module', 'Smooth.muscle.cell.module', 'ER.stress.module', 'IFNg.signaling.modul

Rearrange the input so that cell cycle genes are first

In [19]:
def rearrange_input(adata, gene_name):
  """
  Rearranges the columns of adata, so that cell cycling genes appear first and non-cell
  cycling genes appear after.
  gene_name: A string that is the name of the variable in adata.var containing the gene name.
  """
  cycle_gene = pd.read_csv('/content/drive/MyDrive/mvae/data/scdata/cycle_gene.txt', header=None, sep='\t')
  print(f"Number of total cell cycle genes: {len(cycle_gene[0])}")
  print(cycle_gene[0])
  # Genes involved with the cell cycle present in adata
  cycle_gene = cycle_gene[0][cycle_gene[0].isin(adata.var[gene_name])]

  contained = adata.var[gene_name].isin(cycle_gene)
  print(f"Number of cell cycle genes present in adata: {np.sum(contained)}")

  # cycle_indices = []
  # non_cycle_indices = []
  cycle_g = []
  cycle_indices = np.zeros(np.sum(contained))
  non_cycle_indices = np.zeros(len(contained)-np.sum(contained))
  non_cycle_g = []

  count = 0
  count_1 = 0
  for i in range(0, len(contained)):
    if contained.iloc[i]:
      cycle_indices[count]=i
      cycle_g.append(adata.var[gene_name].iloc[i])
      count = count + 1
    else:
      non_cycle_indices[count_1] = i
      non_cycle_g.append(adata.var[gene_name].iloc[i])
      count_1 = count_1 + 1

  new_indices = np.hstack((cycle_indices, non_cycle_indices)).astype(int)
  print(new_indices)
  # Place the cycling genes first
  # cycling_columns = adata.X[:, cycle_indices]
  # Place the non-cycling genes after
  # non_cycling_columns = adata.X[:, non_cycle_indices]

  # new_X = np.hstack((cycling_columns, non_cycling_columns))
  new_primerid = np.array(cycle_g + non_cycle_g)

  adata.obsm['X_unrearranged'] = adata.X
  adata.X = adata.X[:, new_indices]
  adata.var[gene_name + '_unrearranged'] = adata.var[gene_name]
  adata.var[gene_name] = new_primerid

  return cycle_gene

In [20]:
gene_name = 'gene_name'
rearrange = True
if rearrange:
  print("Rearranging so that cell cycle genes appear first and the non-cell cycle genes follow.")
  cycle_gene = rearrange_input(adata, gene_name)
  adata.var[gene_name].index = np.arange(0, adata.n_vars, 1)

Rearranging so that cell cycle genes appear first and the non-cell cycle genes follow.
Number of total cell cycle genes: 800
0          ABCA7
1            ACD
2          ACYP1
3        ADAMTS1
4          ADCK2
         ...    
795        ZNF98
796        ZNFX1
797    ZNFX1-AS1
798         ZPBP
799      ZSCAN5A
Name: 0, Length: 800, dtype: object
Number of cell cycle genes present in adata: 729
[   86   216   234 ... 36598 36599 36600]


In [21]:
adata.var[gene_name]

0              GNB1
1              RERE
2            PIK3CD
3             TMCO4
4            HP1BP3
            ...    
36596    AC141272.1
36597    AC023491.2
36598    AC007325.1
36599    AC007325.4
36600    AC007325.2
Name: gene_name, Length: 36601, dtype: object

In [22]:
adata.var[gene_name+'_unrearranged']

MIR1302-2HG    MIR1302-2HG
FAM138A            FAM138A
OR4F5                OR4F5
AL627309.1      AL627309.1
AL627309.3      AL627309.3
                  ...     
AC141272.1      AC141272.1
AC023491.2      AC023491.2
AC007325.1      AC007325.1
AC007325.4      AC007325.4
AC007325.2      AC007325.2
Name: gene_name_unrearranged, Length: 36601, dtype: object